In [33]:
import requests
import pandas as pd


In [65]:
url = "https://restcountries.com/v3.1/all"


In [66]:
def tentar_conexao(url):

    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException:
        return None

In [70]:

data = tentar_conexao(url)
if data is not None:
    print("Dados recebidos:")
    print(data[0])
else:
    print("Falha ao acessar dados do site.")



Falha ao acessar dados do site.


In [37]:
table1 = pd.DataFrame([{
    "Nome Comum": country["name"]["common"],
    "Nome Oficial": country["name"]["official"],
    "Código Alpha-2": country["cca2"],
    "Região": country["region"],
    "Latitude": country["latlng"][0] if country["latlng"] else None,
    "Longitude": country["latlng"][1] if country["latlng"] else None,
    "Capital": country["capital"][0] if country.get("capital") else None
} for country in data])
table1

,Nome Comum,Nome Oficial,Código Alpha-2,Região,Latitude,Longitude,Capital
0,South Georgia,South Georgia and the South Sandwich Islands,GS,Antarctic,-54.500000,-37.000000,King Edward Point
1,Grenada,Grenada,GD,Americas,12.116667,-61.666667,St. George's
2,Switzerland,Swiss Confederation,CH,Europe,47.000000,8.000000,Bern
3,Sierra Leone,Republic of Sierra Leone,SL,Africa,8.500000,-11.500000,Freetown
4,Hungary,Hungary,HU,Europe,47.000000,20.000000,Budapest
...,...,...,...,...,...,...,...
245,Belgium,Kingdom of Belgium,BE,Europe,50.833333,4.000000,Brussels
246,Israel,State of Israel,IL,Asia,31.470000,35.130000,Jerusalem
247,New Zealand,New Zealand,NZ,Oceania,-41.000000,174.000000,Wellington
248,Nicaragua,Republic of Nicaragua,NI,Americas,13.000000,-85.000000,Managua


In [38]:
table2 = pd.DataFrame([{
    "Nome Comum": country["name"]["common"],
    "População": country["population"],
    "Área": country["area"],
    "Continente": country["continents"][0],
    "País sem litoral": country["landlocked"]
} for country in data])
table2

,Nome Comum,População,Área,Continente,País sem litoral
0,South Georgia,30,3903.0,Antarctica,False
1,Grenada,112519,344.0,North America,False
2,Switzerland,8654622,41284.0,Europe,True
3,Sierra Leone,7976985,71740.0,Africa,False
4,Hungary,9749763,93028.0,Europe,True
...,...,...,...,...,...
245,Belgium,11555997,30528.0,Europe,False
246,Israel,9216900,20770.0,Asia,False
247,New Zealand,5084300,270467.0,Oceania,False
248,Nicaragua,6624554,130373.0,North America,False


In [39]:
table3 = pd.DataFrame([{
    "Nome Comum": country["name"]["common"],
    "Independente": country.get("independent"),
    "Membro da ONU": country.get("unMember"),
    "Status": country.get("status")
} for country in data])
table3

,Nome Comum,Independente,Membro da ONU,Status
0,South Georgia,False,False,officially-assigned
1,Grenada,True,True,officially-assigned
2,Switzerland,True,True,officially-assigned
3,Sierra Leone,True,True,officially-assigned
4,Hungary,True,True,officially-assigned
...,...,...,...,...
245,Belgium,True,True,officially-assigned
246,Israel,True,True,officially-assigned
247,New Zealand,True,True,officially-assigned
248,Nicaragua,True,True,officially-assigned
